In [63]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact,fixed

# Simpson's Rule

Simpson's rule is a means of approximating a integral integral $$ I = \int_a^b f(x)\,dx$$ using only a finite number of values of $f$.

More specifically, let $n$ be a positive, even integer. We partition the interval $[a,b]$ into $n$ uniform subintervals of width $$\Delta x = \frac{b-a}{n}$$.

\begin{align*}
x_0 &= a \\
x_1 &= a + \Delta x \\
\ &\vdots \\
x_i &= a  + i\Delta x \\
\ &\vdots \\
x_n &= b
\end{align*}


Since $n$ is even, we take these subintervals 2 at a time. Then we fit a quadratic function to the three function values at the edges of this pair of intervals. Graphically,

In [61]:
@interact
def _(y0=(0,3.),y1=(0,4.),y2=(0,3.)):
    plt.plot([-1,-1],[0,y0],'--k')
    plt.plot([0,0],[0,y1],'--k')
    plt.plot([1,1],[0,y2],'--k')
    x = np.linspace(-1,1,55)
    A = np.array([[1,-1,1],[1,0,0],[1,1,1]])
    c,b,a = np.linalg.solve(A,[y0,y1,y2])
    plt.plot(x,a*x**2 + b*x + c,'k')
    plt.fill(np.concatenate([x,[x[-1],x[0]]]),np.concatenate([a*x**2 + b*x + c,[0,0]]),alpha=.5)
    plt.ylim([0,4])
    plt.xticks([-1,0,1],['$-h$','$0$','$h$'])

interactive(children=(FloatSlider(value=1.5, description='y0', max=3.0), FloatSlider(value=2.0, description='y…

Since we don't actually care about the expression for this curve, only its integral, we can make a shortcut. Suppose the three points are translated to $-h,0,h$ and let $ax^2 + bx+c$ be the curve function that fits these values, so 

\begin{align*}
y_0 &= a(-h)^2 + b(-h) + c  \\
y_1 &= c  \\
y_2 &= a(h)^2 + b(h) + c  \\
\end{align*}

Now note we have $$y_0 + 4y_1 +y_2 = 2ah^2 +6c$$ (you'll see why this matters in a second).

We integrate 

$$\int_{-h}^h ax^2+bx+c\,dx = 2 a\frac{h^3}{3} + 2ch = \frac{h}{3}(2ah^2 +6c) $$ $$= \frac{h}{3}(y_0 + 4 y_1 +y_2)$$

So the area under each pair of subintervals can be computed this way, and when we add them together, the terms on the "inside edge" of each pair gets added twice. Thus a pattern emerges.

\begin{align*}
I \approx \frac{\Delta x}{3} (&f(x_0) + 4f(x_1) + f(x_2) \  \\
+ &f(x_2) + 4f(x_3) + f(x_4) \\
 &\vdots \\
+ &f(x_{n-2}) + 4f(x_{n-1}) + f(x_{n}) )
\end{align*}
$$ = \frac{\Delta x}{3} (f(x_0) + 4f(x_1) + 2f(x_2) + 4f(x_3) +\cdots + 2f(x_{n-2}) + 4f(x_{n-1}) + f(x_{n}) ) $$

This last estimate is what is known as **Simpson's Rule**.

#### Example 

Estimate the definite integral $$\int_0^\pi x^2 \sin^2{x}\,dx $$ using Simpson's Rule. 

In [85]:
@interact
def _(n=(2,30,2),f=fixed(lambda x: x**2*np.sin(x)**2)):
    for i in range(0,n,2):
        x0,x1,x2 = [(np.pi/n*j) for j in range(i,i+3)]
        y0,y1,y2 = [f(x0),f(x1),f(x2)]
        plt.plot([x0,x0],[0,y0],'--k')
        plt.plot([x1,x1],[0,y1],'--k')
#         plt.plot([1,1],[0,y2],'--k')
        x = np.linspace(x0,x2,25)
        A = np.array([[1,x0,x0**2],[1,x1,x1**2],[1,x2,x2**2]])
        c,b,a = np.linalg.solve(A,[y0,y1,y2])
        plt.plot(x,a*x**2 + b*x + c,'k')
        plt.fill(np.concatenate([x,[x[-1],x[0]]]),np.concatenate([a*x**2 + b*x + c,[0,0]]),alpha=.6)
    x = np.linspace(0,np.pi,55)
    plt.plot(x,f(x),'k')
    plt.fill(np.concatenate([x,[x[-1],x[0]]]),np.concatenate([f(x),[0,0]]),'r',alpha=.15)
    plt.ylim([0,3.5])
    plt.xticks([np.pi/4*i for i in range(5)],["$\\frac{{ {} \pi }}{{4}}$".format(i) for i in range(5)])

interactive(children=(IntSlider(value=16, description='n', max=30, min=2, step=2), Output()), _dom_classes=('w…

In [86]:
def sr_vector(n):
    """This is a generator for producing the 142424...4241 pattern in Simpson's Rule"""
    yield 1
    for _ in range(n//2-1):
        yield 4
        yield 2
    yield 4
    yield 1

def simpsons_rule(f,a,b,n=10):
    """Approximates the definite integral of f from a to b using n 
    subintervals via Simpson's Rule. 
    n must be even.
    """
    dx = (b-a)/(n)
    return dx/3*np.dot(list(sr_vector(n)),[f(a+i*dx) for i in range(n+1)])

In [76]:
from scipy.integrate import quad
quad(lambda x: x**2*np.sin(x)**2,0,np.pi)[0]


4.382314616652521

In [87]:
for n in range(2,25,2):
    print(f"{n:2d}",simpsons_rule(lambda x: x**2*np.sin(x)**2,0,np.pi,n))
print("  ",II[0])

 2 5.167712780049969
 4 4.521748682543723
 6 4.402125701524049
 8 4.387965137547131
10 4.384524331585615
12 4.383354231493331
14 4.382867548618777
16 4.382635661414043
18 4.382513743235085
20 4.382444658492079
22 4.382403133008028
24 4.382376952596762
   4.382314616652521


In [29]:
# Here's another example.
# quad(lambda x : 4/(1+x**2),0,1)[0]
simpsons_rule(lambda x : 4/(1+x**2),0,1,1000)

3.141592653589793